In [30]:
import pandas as pd
demographic_df = pd.read_csv("../data/processed/interim/demographic_raw_merged.csv")


In [31]:
demographic_df.head()

,date,state,district,pincode,demo_age_5_17,demo_age_17_
0,01-03-2025,Uttar Pradesh,Gorakhpur,273213,49,529
1,01-03-2025,Andhra Pradesh,Chittoor,517132,22,375
2,01-03-2025,Gujarat,Rajkot,360006,65,765
3,01-03-2025,Andhra Pradesh,Srikakulam,532484,24,314
4,01-03-2025,Rajasthan,Udaipur,313801,45,785


In [32]:
demographic_df.columns

Index(['date', 'state', 'district', 'pincode', 'demo_age_5_17',
       'demo_age_17_'],
      dtype='object')

In [33]:
demographic_df.shape

(2071700, 6)

In [34]:
demographic_df["state"] = demographic_df["state"].str.strip().str.title()
demographic_df["district"] = demographic_df["district"].str.strip().str.title()

In [35]:
demographic_df = demographic_df.rename(columns={
    "demo_age_17_": "demo_age_17_plus"
})


In [36]:
demographic_df["date"] = pd.to_datetime(demographic_df["date"], dayfirst=True)

In [37]:
demographic_df[
    demographic_df["district"].isin(["100000", 100000])
].shape[0]


2

In [38]:
demographic_df[
    demographic_df["state"].isin(["100000", 100000])
].shape[0]


2

In [39]:
demographic_df = demographic_df[
    ~(
        demographic_df["district"].isin(["100000", 100000]) |
        demographic_df["state"].isin(["100000", 100000])
    )
]

In [40]:
demographic_df.shape

(2071698, 6)

In [41]:
demographic_df = demographic_df.drop_duplicates()

In [42]:
demographic_df.shape

(1598010, 6)

In [43]:
demographic_df.isnull().sum()


date                0
state               0
district            0
pincode             0
demo_age_5_17       0
demo_age_17_plus    0
dtype: int64

In [44]:
demographic_df.to_csv(
    "../data/processed/cleaned/demographic_clean.csv",
    index=False
)


In [45]:
demographic_df["state"].value_counts()


state
Andhra Pradesh                              154789
Tamil Nadu                                  153333
Uttar Pradesh                               132366
Maharashtra                                 126303
West Bengal                                 124762
Karnataka                                   120403
Kerala                                       82186
Gujarat                                      75161
Bihar                                        74536
Odisha                                       71732
Telangana                                    70098
Rajasthan                                    68455
Madhya Pradesh                               60157
Assam                                        47408
Punjab                                       38097
Jharkhand                                    31225
Chhattisgarh                                 27286
Haryana                                      22341
Himachal Pradesh                             21940
Uttarakhand              

In [46]:
state_mapping = {
    # West Bengal
    "West Bangal": "West Bengal",
    "Westbengal": "West Bengal",
    "West  Bengal": "West Bengal",
    "West Bengli": "West Bengal",

    # Odisha
    "Orissa": "Odisha",

    # J&K
    "Jammu & Kashmir": "Jammu And Kashmir",

    # Puducherry
    "Pondicherry": "Puducherry",

    # UT mergers
    "Dadra And Nagar Haveli": "Dadra And Nagar Haveli And Daman And Diu",
    "Dadra & Nagar Haveli": "Dadra And Nagar Haveli And Daman And Diu",
    "Daman And Diu": "Dadra And Nagar Haveli And Daman And Diu",
    "Daman & Diu": "Dadra And Nagar Haveli And Daman And Diu",

    # Andaman
    "Andaman & Nicobar Islands": "Andaman And Nicobar Islands",

    # Uttarakhand
    "Uttaranchal": "Uttarakhand",

    # Chhattisgarh
    "Chhatisgarh": "Chhattisgarh"
}

In [47]:
demographic_df["state"] = demographic_df["state"].replace(state_mapping)


In [48]:
state_correction = {
    "Darbhanga": "Bihar",
    "Jaipur": "Rajasthan",
    "Nagpur": "Maharashtra",
    "Puttenahalli": "Karnataka",
    "Balanagar": "Telangana",
    "Madanapalle": "Andhra Pradesh",
    "Raja Annamalai Puram": "Tamil Nadu"
}


In [49]:
demographic_df["state"] = demographic_df["state"].replace(state_correction)


In [58]:
telangana_districts = [
    "Adilabad",
    "Hyderabad",
    "K.V.Rangareddy",
    "K.V. Rangareddy",
    "Karim Nagar",
    "Karimnagar",
    "Khammam",
    "Mahabub Nagar",
    "Mahabubnagar",
    "Mahbubnagar",
    "Medak",
    "Nalgonda",
    "Nizamabad",
    "Rangareddi",
    "Warangal",
    "Y.S.R. Kadapa"
    # add more if needed
]


mask = (
    demographic_df["district"]
    .astype(str)
    .str.strip()
    .str.title()
    .isin(telangana_districts)
)

affected_rows = mask.sum()

demographic_df.loc[mask, "state"] = "Telangana"

print(f"✔ Telangana correction applied to {affected_rows} rows")

✔ Telangana correction applied to 72551 rows


In [51]:
# -----------------------------
# Fix Kamrup → Assam
# -----------------------------

mask = (
    demographic_df["district"].astype(str).str.strip().str.title() == "Kamrup"
)

affected_rows = mask.sum()

demographic_df.loc[mask, "state"] = "Assam"

print(f"✔ Kamrup correction applied to {affected_rows} rows")

✔ Kamrup correction applied to 2716 rows


In [52]:
## -----------------------------
# Fix Leh → Ladakh
# -----------------------------

mask = demographic_df["district"].astype(str).str.strip().str.title().isin(
    ["Leh", "Leh(Ladakh)"]
)

affected_rows = mask.sum()
demographic_df.loc[mask, "state"] = "Ladakh"

print(f"✔ Leh correction applied to {affected_rows} rows")

✔ Leh correction applied to 568 rows


In [53]:
## -----------------------------
# Fix Leh → Ladakh
# -----------------------------

mask = (
    demographic_df["district"].astype(str).str.strip().str.title() == "Leh"
)

affected_rows = mask.sum()
demographic_df.loc[mask, "state"] = "Ladakh"

print(f"✔ Leh correction applied to {affected_rows} rows")

✔ Leh correction applied to 568 rows


In [54]:
## -----------------------------
# Fix Leh → Ladakh
# -----------------------------

mask = (
    demographic_df["district"].astype(str).str.strip().str.title() == "Leh (Ladakh)"
)

affected_rows = mask.sum()
demographic_df.loc[mask, "state"] = "Ladakh"

print(f"✔ Leh correction applied to {affected_rows} rows")

✔ Leh correction applied to 6 rows


In [55]:
demographic_df["state"].value_counts()


state
Tamil Nadu                                  153334
Uttar Pradesh                               132366
Maharashtra                                 126304
West Bengal                                 125013
Karnataka                                   120404
Andhra Pradesh                              115732
Telangana                                   109157
Odisha                                       82463
Kerala                                       82186
Gujarat                                      75161
Bihar                                        74537
Rajasthan                                    68456
Madhya Pradesh                               60157
Assam                                        47408
Punjab                                       38097
Jharkhand                                    31225
Chhattisgarh                                 27288
Haryana                                      22341
Himachal Pradesh                             21940
Uttarakhand              

In [56]:
# Remove rows with <unset> in key location columns

before = len(demographic_df)

demographic_df = demographic_df[
    (demographic_df["state"] != "<unset>") &
    (demographic_df["district"] != "<unset>") &
    (demographic_df["pincode"] != "<unset>")
]

after = len(demographic_df)

print(f"Removed {before - after} rows with <unset> in key columns")


Removed 0 rows with <unset> in key columns


In [57]:
demographic_df.to_csv(
    "../data/processed/cleaned/demographic_clean.csv",
    index=False
)
